# Parabricks Hands-On Workshop

## Tumor Sequencing: Somatic Variant calling Workflow

For tumor sequencing, there will be several analysis to perform:
- Read alignment of tumor sequencing sample
- Read alignment of normal sequencing sample (if doing tumor-normal paired analysis)
- Somatic variant calling
- RNA read alignment (for calling gene fusions from RNA-seq)
- Gene fusion detection

\We will start from downloading the reference genome and sample files, and then proceed through the analysis step-by-step.

#### GPU Monitoring

In [7]:
!nvidia-smi

Fri Oct 11 03:09:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB-N         On  | 00000000:0A:00.0 Off |                    0 |
| N/A   35C    P0              44W / 160W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Run the command below in the terminal
### watch -n 0.5 nvidia-smi
#

#### Download the Reference Genome

We will download the sample data from the germline workflow tutorial, which includes the reference genome and its index files.

In [ ]:
# The tar file is 9.3GB and, when extracted, an additional 14GB
!mkdir sample_data
%cd sample_data
!wget -O parabricks_sample.tar.gz "https://s3.amazonaws.com/parabricks.sample/parabricks_sample.tar.gz"
!tar xvf parabricks_sample.tar.gz
!mv parabricks_sample/* .
%cd ..

In [6]:
!ls sample_data/Ref

Homo_sapiens_assembly38.dict
Homo_sapiens_assembly38.fasta
Homo_sapiens_assembly38.fasta.amb
Homo_sapiens_assembly38.fasta.ann
Homo_sapiens_assembly38.fasta.bwt
Homo_sapiens_assembly38.fasta.fai
Homo_sapiens_assembly38.fasta.pac
Homo_sapiens_assembly38.fasta.sa
Homo_sapiens_assembly38.known_indels.vcf.gz
Homo_sapiens_assembly38.known_indels.vcf.gz.tbi


In [11]:
!mkdir outputdir

#### Download Sample Data Using SRAtoolkit

The SeqC2 consortium did a series of studies to o establish best practices, reference standards, and benchmark the results of somatic mutation detections under different bioinformatic and laboratory conditions (https://sites.google.com/view/seqc2 paper: https://www.nature.com/articles/s41587-021-00993-6). Download a tumor-normal paired sequencing data for analsis.

Download SRAtoolkit and download the sequencing files using it.

In [2]:
## Download SRAtoolkit to process SRA files
!wget --output-document sratoolkit.tar.gz https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-centos_linux64.tar.gz
!tar -vxzf sratoolkit.tar.gz

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/tutorial/sample_data
--2024-10-16 02:32:48--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-centos_linux64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.7, 2607:f220:41e:250::10, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93535881 (89M) [application/x-gzip]
Saving to: ‘sratoolkit.tar.gz’

sratoolkit.tar.gz   100%[===================>]  89.20M  33.6MB/s    in 2.7s    

2024-10-16 02:32:51 (33.6 MB/s) - ‘sratoolkit.tar.gz’ saved [93535881/93535881]

sratoolkit.3.1.1-centos_linux64/
sratoolkit.3.1.1-centos_linux64/README.md
sratoolkit.3.1.1-centos_linux64/README-vdb-config
sratoolkit.3.1.1-centos_linux64/schema/
sratoolkit.3.1.1-centos_linux64/schema/vdb/
sratoolkit.3.1.1-centos_linux64/schema/vdb/vdb.vschema
sratoolkit.3.1.1-centos_linux64/schema/vdb/built-in.vschema
sratoolkit.3.1.1-centos_

Add sratoolkit executable into the PATH for convenience. Check to make sure we are able to use the tool `fastq-dump`.

In [2]:
import os
sra_bin_path = os.popen("cd sample_data/sratoolkit.*/bin && pwd").read().strip()
os.environ["PATH"] += os.pathsep + sra_bin_path
!which fastq-dump

/tutorial/sample_data/sratoolkit.3.1.1-centos_linux64/bin/fastq-dump


Check that `fastq-dump` works by running the following command. In a few seconds, you should see 10 lines of output.

In [4]:
#Check that sratoolkit is properly installed and fastq-dump works
!fastq-dump --stdout -X 2 SRR390728

Read 2 spots for SRR390728
Written 2 spots for SRR390728
@SRR390728.1 1 length=72
CATTCTTCACGTAGTTCTCGAGCCTTGGTTTTCAGCGATGGAGAATGACTTTGACAAGCTGAGAGAAGNTNC
+SRR390728.1 1 length=72
;;;;;;;;;;;;;;;;;;;;;;;;;;;9;;665142;;;;;;;;;;;;;;;;;;;;;;;;;;;;;96&&&&(
@SRR390728.2 2 length=72
AAGTAGGTCTCGTCTGTGTTTTCTACGAGCTTGTGTTCCAGCTGACCCACTCCCTGGGTGGGGGGACTGGGT
+SRR390728.2 2 length=72
;;;;;;;;;;;;;;;;;4;;;;3;393.1+4&&5&&;;;;;;;;;;;;;;;;;;;;;<9;<;;;;;464262


Download sequencing files. SRR7890899 is a whole-exome sequencing for the tumor breast cell line HCC1395. SRR7890845 is a whole-exome sequencing for the matched normal HCC1395BL B lymphocyte cell line. These files are quite large and took hours to download.

In [12]:
%cd sample_data/Data
!fastq-dump --split-files --gzip SRR7890845
!fastq-dump --split-files --gzip SRR7890899
%cd ../..

Read 57058265 spots for SRR7890845
Written 57058265 spots for SRR7890845


Download whole-genome sequencing files of a tumor and its matched normal sample. The files are large and took a long time to download.

In [ ]:
## Download publicly available SRA files using wget. Both files are
# about 65 GB in size.
%cd sample_data/Data
# Normal sample
!wget https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR7890827/SRR7890827 --output-document=SRR7890827.sra

# Tumor sample
!wget https://sra-pub-run-odp.s3.amazonaws.com/sra/SRR7890824/SRR7890824 --output-document=SRR7890824.sra

## Convert SRA to FASTQ files
!fastq-dump --split-files ./SRR7890827.sra --gzip
!fastq-dump --split-files ./SRR7890824.sra --gzip
%cd ../..

#### Run Somatic Workflow: BWA-MEM + Mutect2

The Somatic Workflow is one command that analyzes tumor sequencing from raw FASTQ file to somatic variants in VCF format. It performs alignment using BWA-MEM on both tumor and normal sequencing files and uses Mutect2 to generate a single VCF file containing somatic variants. If you wish to perform tumor-only analysis, just omit the `in-normal-fq` and `out-normal-bam` options.

In [41]:
!pbrun somatic \
    --ref sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-normal-fq sample_data/Data/SRR7890845_1.fastq.gz sample_data/Data/SRR7890845_2.fastq.gz "@RG\tID:SRR7890845_rg1\tLB:lib_normal\tPL:ILLUMINA\tSM:normal_sample\tPU:SRR7890845_rg1" \
    --in-tumor-fq sample_data/Data/SRR7890899_1.fastq.gz sample_data/Data/SRR7890899_2.fastq.gz "@RG\tID:SRR7890899_rg1\tLB:lib_tumor\tPL:ILLUMINA\tSM:tumor_sample\tPU:SRR7890899_rg1" \
    --bwa-options="-Y" \
    --out-normal-bam outputdir/normal.bam \
    --out-tumor-bam outputdir/tumor.bam \
    --out-vcf outputdir/somatic_variants.vcf \
    --low-memory \
    --mutect-low-memory \
    --num-gpus 1

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation


[Parabricks Options Mesg]: Read group created for /tutorial/sample_data/Data/SRR7890899_1.fastq.gz and
/tutorial/sample_data/Data/SRR7890899_2.fastq.gz
[Parabricks Options Mesg]: @RG\tID:SRR7890899_rg1\tLB:lib_tumor\tPL:ILLUMINA\tSM:tumor_sample\tPU:SRR7890899_rg1
[Parabricks Options Mesg]: Read group created for /tutorial/sample_data/Data/SRR7890845_1.fastq.gz and
/tutorial/sample_data/Data/SRR7890845_2.fastq.gz
[Parabricks Options Mesg]: @RG\tID:SRR7890845_rg1\tLB:lib_normal\tPL:ILLUMINA\tSM:normal_sample\tPU:SRR7890845_rg1


[Parabricks Options Mesg]: Checking argument compatibility
[Parabricks Options Mesg]: Set --bwa-options="-K #" to produce compatible pair-ended results with previous versions of
fq2bam or BWA MEM.
[Parabricks Options Mesg]: Read group created for /tutorial/sample_data/Data/SRR7890899_1.fastq.gz and
/tutorial/sample_data/Data/SRR7890899_2.fastq.gz
[Parabricks Options Mesg]: @RG\tI

#### Run Alignment and Variant Calling Separately

The alignment and variant calling steps within the somatic workflow can be run separately. Remember to run fq2bam on the tumor fastq and normal fastqa separately. Variant calling can be done by Mutext2 or DeepSomatic.

- Mutect2

In [ ]:
!pbrun mutectcaller \
    --ref sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-normal-bam outputdir/normal.bam \
    --in-tumor-bam outputdir/tumor.bam \
    --out-vcf outputdir/somatic_variants.vcf \
    --mutect-low-memory \
    --num-gpus 1

- DeepSomatics

In [ ]:
!pbrun deepsomatic \
    --ref sample_data/Ref/Homo_sapiens_assembly38.fasta \
    --in-normal-bam outputdir/normal.bam \
    --in-tumor-bam outputdir/tumor.bam \
    --out-vcf outputdir/somatic_variants.vcf \

## Gene Fusion Detection

#### Download Sample Dataset

In [10]:
## Download STAR-Fusion benchmark dataset breast cancer cell line  BT474
%cd sample_data/Data
!wget https://zenodo.org/records/13363154/files/BT474.Left.fq.gz
!wget https://zenodo.org/records/13363154/files/BT474.Right.fq.gz
%cd ../..

--2024-10-18 07:02:06--  https://zenodo.org/records/13363154/files/BT474.Left.fq.gz
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067370758 (1018M) [application/octet-stream]
Saving to: ‘BT474.Left.fq.gz’

BT474.Left.fq.gz    100%[===================>]   1018M  17.0MB/s    in 57s     

2024-10-18 07:03:04 (18.0 MB/s) - ‘BT474.Left.fq.gz’ saved [1067370758/1067370758]

--2024-10-18 07:03:04--  https://zenodo.org/records/13363154/files/BT474.Right.fq.gz
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1078442985 (1.0G) [application/octet-stream]
Saving to: ‘BT474.Right.fq.gz’

BT474.Right.fq.gz   100%[===================>]   1.00G  18.4MB/s    in 57s   

In [19]:
## Download STAR-Fusion benchmark dataset prostate cancer cell line VCaP
%cd sample_data/Data

!wget https://zenodo.org/records/13363154/files/SRR1217085_1.fastq.gz.20M.fq.gz
!wget https://zenodo.org/records/13363154/files/SRR1217085_2.fastq.gz.20M.fq.gz
%cd ../..

--2024-10-19 08:03:24--  https://zenodo.org/records/13363154/files/SRR1217085_1.fastq.gz.20M.fq.gz
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2158029851 (2.0G) [application/octet-stream]
Saving to: ‘SRR1217085_1.fastq.gz.20M.fq.gz’

SRR1217085_1.fastq. 100%[===================>]   2.01G  18.2MB/s    in 1m 56s  

2024-10-19 08:05:21 (17.8 MB/s) - ‘SRR1217085_1.fastq.gz.20M.fq.gz’ saved [2158029851/2158029851]

--2024-10-19 08:05:21--  https://zenodo.org/records/13363154/files/SRR1217085_2.fastq.gz.20M.fq.gz
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2048873466 (1.9G) [application/octet-stream]
Saving to: ‘SRR1217085_2.fastq.gz.20M.fq.gz’

SRR12

#### Prepare the Reference Genome Index File

Before using Parabricks to accelerate STAR-Fusion for gene fusion detection, the reference genome index file needs to be generated using STAR v2.7.2a run on CPU.

##### Download the CTAT Genome Library

In [19]:
# Download the ctat genome library
%cd sample/Ref
!wget https://data.broadinstitute.org/Trinity/CTAT_RESOURCE_LIB/__genome_libs_StarFv1.10/GRCh38_gencode_v37_CTAT_lib_Mar012021.source.tar.gz
%cd ../..

/usr/bin/sh: 1: cd: can't cd to sample/Ref
--2024-10-18 09:34:02--  https://data.broadinstitute.org/Trinity/CTAT_RESOURCE_LIB/__genome_libs_StarFv1.10/GRCh38_gencode_v37_CTAT_lib_Mar012021.source.tar.gz
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.68.137
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.68.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4184819390 (3.9G) [application/x-gzip]
Saving to: ‘GRCh38_gencode_v37_CTAT_lib_Mar012021.source.tar.gz’

GRCh38_gencode_v37_ 100%[===================>]   3.90G  43.4MB/s    in 92s     

2024-10-18 09:35:35 (43.3 MB/s) - ‘GRCh38_gencode_v37_CTAT_lib_Mar012021.source.tar.gz’ saved [4184819390/4184819390]



Unpack the CTAT genome library and replace the `AnnotFilterRule.pm` file as instructed. 

In [33]:
%cd sample_data/Ref
!tar xvf GRCh38_gencode_v37_CTAT_lib_Mar012021.source.tar.gz 
%cd GRCh38_gencode_v37_CTAT_lib_Mar012021.source
!wget https://data.broadinstitute.org/Trinity/CTAT_RESOURCE_LIB/__genome_libs_StarFv1.10/AnnotFilterRule.pm
%cd ../../..

/tutorial/sample_data/Ref


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


GRCh38_gencode_v37_CTAT_lib_Mar012021.source/
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/fusion_lib.Mar2021.dat.gz
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/GRCh38.primary_assembly.genome.fa.pseudo_masked.fa
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/customized_gtf.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/ref_annot.cdsplus.fa.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/pseudo_mask_genome.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/ref_annot.cdna.fa.allvsall.blastn.outfmt6.toGenes.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/ref_annot.cdsplus.dfam_masked.fa.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/ref_annot.cdsplus.dfam_masked.fa.blidx.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/ref_annot.cdsplus.dfam_masked.fa.allvsall.outfmt6.genesym.best.gz.ok
GRCh38_gencode_v37_CTAT_lib_Mar012021.source/__loc_chkpts/

##### Download STAR 

Parabricks has been tested to be compatible with STAR 2.7.2a, so we will download this version of STAR to generate the reference genome index file.

In [22]:
# Download STAR to make the genome index file for rna_fq2bam
!wget https://github.com/alexdobin/STAR/archive/2.7.2a.tar.gz
!tar -xzf 2.7.2a.tar.gz
%cd STAR-2.7.2a/source
!make STAR

--2024-10-19 09:18:28--  https://github.com/alexdobin/STAR/archive/2.7.2a.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/alexdobin/STAR/tar.gz/refs/tags/2.7.2a [following]
--2024-10-19 09:18:28--  https://codeload.github.com/alexdobin/STAR/tar.gz/refs/tags/2.7.2a
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘2.7.2a.tar.gz’

2.7.2a.tar.gz           [      <=>           ]   7.76M  6.99MB/s    in 1.1s    

2024-10-19 09:18:30 (6.99 MB/s) - ‘2.7.2a.tar.gz’ saved [8142240]

make: *** No rule to make target 'STAR'.  Stop.


Once STAR is installed, set the PATH environmental variable for convenience.

In [3]:
STAR_path = os.popen("cd STAR-2.7.2a/source && pwd").read().strip()
os.environ["PATH"] += os.pathsep + STAR_path
!echo $PATH

/usr/local/parabricks:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tutorial/sample_data/sratoolkit.3.1.1-centos_linux64/bin:/tutorial/STAR-2.7.2a/source


##### Generate Reference Genome Index

Run STAR 2.7.2a to generate the reference genome index file. The read length of SRR1217085 is 99, so `--sjdbOverhang `is set to 100 (read length plus one). This step is time consuming and took 40 min on my machine (80 CPU cores, 520 GB RAM).

In [34]:
!STAR --help

Usage: STAR  [options]... --genomeDir REFERENCE   --readFilesIn R1.fq R2.fq
Spliced Transcripts Alignment to a Reference (c) Alexander Dobin, 2009-2019

For more details see:
<https://github.com/alexdobin/STAR>
<https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf>

### versions
versionGenome           2.7.1a
    string: earliest genome index version compatible with this STAR release. Please do not change this value!

### Parameter Files
parametersFiles          -
    string: name of a user-defined parameters file, "-": none. Can only be defined on the command line.

### System
sysShell            -
    string: path to the shell binary, preferably bash, e.g. /bin/bash.
                    - ... the default shell is executed, typically /bin/sh. This was reported to fail on some Ubuntu systems - then you need to specify path to bash.

### Run Parameters
runMode                         alignReads
    string: type of the run.

                                alignReads         

In [8]:
## Prepare the reference genome index file using STAR
!STAR --runMode genomeGenerate \
     --genomeDir /tutorial/sample_data/Ref/GRCh38_gencode_v37_CTAT_lib_Mar012021.source/ \
     --genomeFastaFiles /tutorial/sample_data/Ref/GRCh38_gencode_v37_CTAT_lib_Mar012021.source/GRCh38.primary_assembly.genome.fa \
     --sjdbGTFfile /tutorial/sample_data/Ref/GRCh38_gencode_v37_CTAT_lib_Mar012021.source/gencode.v37.annotation.gtf \
     --sjdbOverhang 100 \
     --runThreadN 16

Oct 20 06:51:36 ..... started STAR run
Oct 20 06:51:36 ... starting to generate Genome files
Oct 20 06:53:15 ... starting to sort Suffix Array. This may take a long time...
Oct 20 06:53:42 ... sorting Suffix Array chunks and saving them to disk...
Oct 20 07:49:02 ... loading chunks from disk, packing SA...
Oct 20 07:50:51 ... finished generating suffix array
Oct 20 07:50:51 ... generating Suffix Array index
Oct 20 07:55:07 ... completed Suffix Array index
Oct 20 07:55:07 ..... processing annotations GTF
Oct 20 07:55:33 ..... inserting junctions into the genome indices
Oct 20 08:01:19 ... writing Genome to disk ...
Oct 20 08:01:24 ... writing Suffix Array to disk ...
Oct 20 08:02:11 ... writing SAindex to disk
Oct 20 08:02:16 ..... finished successfully


#### Run rna_fq2bam for Gene Fusion Detection

In [ ]:
!pbrun rna_fq2bam \
    --genome-lib-dir sample_data/Ref/GRCh38_gencode_v37_CTAT_lib_Mar012021.source \
    --ref sample_data/Ref/GRCh38_gencode_v37_CTAT_lib_Mar012021.source/GRCh38.primary_assembly.genome.fa \
    --in-fq sample_data/Data/SRR1217085_1.fastq.gz.20M.fq.gz sample_data/Data/SRR1217085_2.fastq.gz.20M.fq.gz\
    --out-bam outputdir/rna_fq2bam.bam \
    --output-dir /utputdir 

#### Run Gene Fusion Detection

In [30]:
!pbrun starfusion \
    --output-dir outputdir/starfusion_output \
    --genome-lib-dir sample_data/Ref/GRCh38_gencode_v37_CTAT_lib_Mar012021.source \
    --chimeric-junction outputdir/Chimeric.out.junction

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation

[PB Info 2024-Oct-20 15:49:58] ------------------------------------------------------------------------------
[PB Info 2024-Oct-20 15:49:58] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2024-Oct-20 15:49:58] ||                              Version 4.3.2-1                             ||
[PB Info 2024-Oct-20 15:49:58] ||                                starfusion                                ||
[PB Info 2024-Oct-20 15:49:58] ------------------------------------------------------------------------------
[PB Info 2024-Oct-20 15:49:58] Running starfusion...
[PB Info 2024-Oct-20 15:49:58] Reading "ref_annot.gtf.gene_spans"...
[PB Info 2024-Oct-20 15:49:58] Reading "blast_pairs.idx"...
[PB Info 2024-Oct-20 15:49:58] Reading "trans.blast.align_coords.align_coords.dat"...
[PB Info 2024-Oct-20 15:49:58] Reading reference file...
[PB Info 2024-Oct-20 15:49:58] Reading "fu